In [ ]:
import os
os.chdir("/Users/tien/Documents/PythonEnvs/pytorch/")

: 

In [ ]:
from jup.recsys_models.data.short_video_understanding import read_data
from jup.recsys_models.data.short_video_understanding import process_features

In [ ]:
from jup.recsys_models.features import SparseFeature
from jup.recsys_models.features import DenseFeature

In [ ]:
from jup.recsys_models.core.inputs import get_feature_names

In [ ]:
from jup.recsys_models.models.multi_tasks_multi_towers import NTasksNTowersSharedBottom

In [ ]:
df = read_data()

In [ ]:
df

In [ ]:
sparse_features = ["uid", "user_city", "item_id", "author_id", "item_city", "channel", "music_id", "device"]
dense_features = ["duration_time"]

processed_df = process_features(df, sparse_features, dense_features)

In [ ]:
processed_df

In [ ]:
# Need to figure out what + 1 here
fixlen_feature_columns = ([
    SparseFeature(feat, vocabulary_size=processed_df[feat].max() + 1, embedding_dim=4) for feat in sparse_features] + 
    [DenseFeature(feat, 1, ) for feat in dense_features])

In [ ]:
fixlen_feature_columns

In [ ]:
dnn_feature_columns = fixlen_feature_columns

In [ ]:
dnn_feature_columns

In [ ]:
feature_names = get_feature_names(dnn_feature_columns)

In [ ]:
feature_names

In [ ]:
df.shape

In [ ]:
split_boundary = int(df.shape[0] * 0.8)

In [ ]:
train, test = df[:split_boundary], df[split_boundary:]
# not the train and test contain both values and labels
# thus we need to run the following command to extract only train inputs

train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

In [ ]:
type(train[['finish', 'like']].values)

In [ ]:
type(train_model_input['uid'])

In [ ]:
# Up till now, train model input is dictionary: name -> pandas.core.series.Series

In [ ]:
target = ['finish', 'like']

In [ ]:
model = NTasksNTowersSharedBottom(
    features=dnn_feature_columns,
    task_types=['binary', 'binary'],
    l2_reg_embedding=1e-5,
    task_names=['finish', 'like'],
    device='cpu',
    tensorboard_path="jup/examples/recsys_models/two_tasks_two_towers/runs/"
)

In [ ]:
model.set_optimizer('adagrad')

In [ ]:
model.set_loss_function(loss=["binary_cross_entropy", "binary_cross_entropy"])

In [ ]:
model.set_metrics(metrics=['binary_cross_entropy'])

In [ ]:
model.fit(train_model_input, train[target].values, batch_size=2, epochs=10, verbose=2)

In [ ]:
print(model.feature_col_index)

In [ ]:
print(len(model.tower_dnn_hidden_units))

In [ ]:
model.tower_dnn

In [ ]:
model.outs